# MLP(BitNet)

* https://github.com/kyegomez/BitNet

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/tabledata/binary_classification/bitnet.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install bitnet
%pip install loguru

In [33]:
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


from bitnet import BitLinear

### データセットの用意

In [ ]:
df = sns.load_dataset('titanic')

df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']]
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df.dropna(inplace=True)

X = df.drop('survived', axis=1).values
y = df['survived'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## 通常のNN

In [63]:
class NormalModel(nn.Module):
    def __init__(self):
        super(NormalModel, self).__init__()
        self.fc1 = nn.Linear(6, 64)  # 入力層から隠れ層へ
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)  # 隠れ層から出力層へ

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

normal_model = NormalModel()

## BitNet

In [64]:
class BitNetModel(nn.Module):
    def __init__(self):
        super(BitNetModel, self).__init__()
        self.fc1 = BitLinear(6, 64)
        self.relu = nn.ReLU()
        self.fc2 = BitLinear(64, 2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

bitnet_model = BitNetModel()

### モデルの学習

In [123]:
def train(model, train_loader, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        if (epoch+1) % 10 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            with torch.no_grad():
                correct = 0
                total = 0
                outputs = model(X_test_tensor)
                _, predicted = torch.max(outputs.data, 1)
                total += y_test_tensor.size(0)
                correct += (predicted == y_test_tensor).sum().item()
            print(f'Accuracy: {100 * correct / total:.2f}%')
    return

In [124]:
num_epochs = 100

In [125]:
train(normal_model, train_loader, num_epochs)

Accuracy: 79.72%
Accuracy: 80.42%
Accuracy: 79.02%
Accuracy: 79.02%
Accuracy: 79.02%
Accuracy: 79.72%
Accuracy: 78.32%
Accuracy: 79.02%
Accuracy: 79.02%
Accuracy: 79.72%


In [126]:
train(bitnet_model, train_loader, num_epochs)

Accuracy: 75.52%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 74.13%
Accuracy: 76.22%
Accuracy: 75.52%
